<a href="https://colab.research.google.com/github/drewrya/DS4A---Team-16/blob/main/Flights_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load Google drive where the data and models are stored
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Load packages

import sys
import keras
import numpy as np
import pandas as pd
import random


import datetime
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import resample 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.metrics import classification_report

In [6]:
random.seed(10)

In [3]:
df = pd.read_csv(r'/content/drive/MyDrive/DS4A/2018.csv')
#airports = pd.read_csv(r'/content/drive/MyDrive/DS4A/airports.csv')


In [3]:
#departure delays by airport (there are more than 5K airports in the US)
d_delays_airport = df[['ORIGIN','DEP_DELAY']].groupby(['ORIGIN'])
result = [g[1] for g in list(d_delays_airport))[:3]]
#d_delays_airport.plot.bar()
plt = result.boxplot(subplots=False, rot=45, fontsize=12)
#plt.xlabel('Airport')
#plt.ylabel('# of Delayed Flights')
#plt.title('Delayed Flights by Airport - 2018')
plt

SyntaxError: ignored

In [ ]:
# concatenate years
# frames = [df1, df2, df3]
# result = pd.concat(frames)
dataa = df

In [ ]:
dataa.shape

(7213446, 28)

In [ ]:
 
#dataa = pd.merge(df_airports, airports, on='airport')

In [ ]:
# we have to make the date numeric to be fed into the NN
dataa['FL_DATE'] = pd.DatetimeIndex(dataa['FL_DATE']).month
# geocode airports
dataa = dataa.select_dtypes(['number'])

In [ ]:
dataa['DELAY_FLAG'] = dataa['DEP_DELAY'].gt(15)
dataa['DELAY_FLAG']=dataa['DELAY_FLAG'].astype('uint8')

#dataa['DELAY_FLAG'] = dataa[dataa['DEP_DELAY'] >= 1] = 1

#dataa['DELAY_FLAG'] = dataa[is.na(dataa['DEP_DELAY'])] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
dataa.shape

(7213446, 25)

In [ ]:
(dataa['DELAY_FLAG'] == 0).sum()/7213446

In [ ]:
dataa.DELAY_FLAG.unique()

array([0, 1], dtype=uint8)

In [ ]:
dataa=pd.DataFrame(dataa).fillna(value = -9999999)

In [ ]:
from sklearn.model_selection import StratifiedKFold
#create definitions to prepare the data, splitt the data in train and test and scale the data
def load_dataset():
    dataset = dataa
    X = dataset.iloc[:, 0:23].values
    y = dataset.iloc[:, 24].values
    return X, y
def prepare_train_test_set(X, y):
    skf=StratifiedKFold(n_splits=2)#, random_state=None, shuffle=False)
    for train_index, test_index in skf.split(X, y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    return X_train, X_test, y_train, y_test
def scale_features(X_train, X_test):
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test
#run the data preparation
if __name__ == '__main__':
    labelencoder_x_1 = LabelEncoder()
    dataa.iloc[:, 0] = labelencoder_x_1.fit_transform(dataa.iloc[:, 0])
    dataset_path = sys.argv[1]
    X, y = load_dataset()
    X_train, X_test, y_train, y_test = prepare_train_test_set(X, y)
    X_train, X_test = scale_features(X_train, X_test)

TRAIN: [3601258 3601259 3601260 ... 7213443 7213444 7213445] TEST: [      0       1       2 ... 3635380 3635381 3635386]
TRAIN: [      0       1       2 ... 3635380 3635381 3635386] TEST: [3601258 3601259 3601260 ... 7213443 7213444 7213445]


In [ ]:
#Prepare the earlystop and saved weights
filepath = "model_weights.hdf5"
callbbacks_list = [ModelCheckpoint(filepath, monitor = "val_loss", save_best_only = True, mode = "min")]

In [ ]:
nodes_list2 = [1]
roc_outcomes = []
pr_outcomes = []
loss_hx_list = []
model_number = np.array(np.arange(1, len(nodes_list2)+1)).tolist()
count = 0
columns_list = ["Epochs"]

#Run neural network for all nodes 
import time 
start_time = time.time()

for nodes in nodes_list2:
  count += 1


  columns_list.append("model " + str(count))

  print("\n")
  print("model " + str(count) + "/" + str(len(nodes_list2)))
  print("\n")

  model = Sequential()
  model.add(Dense(units = 23,input_dim=23,activation='relu'))
  model.add(Dense(units = nodes, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
  model.fit(X_train, y_train, validation_split = 0.25, epochs=1, batch_size = 32, callbacks = [EarlyStopping(monitor="val_loss", patience=3)])

  #pred_prob = model.predict_prob(X_test)
  #predictions = model.predict_classes(X_test) #this is the original code
  pred_prob=model.predict(X_test)
  #predictions=np.argmax(predict_prob,axis=1)
  predictions = (model.predict(X_test) > 0.5).astype("int32")

  logit_roc_out = roc_auc_score(y_test, pred_prob)
  average_precision = average_precision_score(y_test, predictions)
  roc_outcomes.append(logit_roc_out)
  pr_outcomes.append(average_precision)

  cm = pd.DataFrame(confusion_matrix(y_test, predictions))
  cm["Total"] = np.sum(cm, axis =1)
  cm = cm.append(np.sum(cm, axis =0), ignore_index =True)
  cm.columns = ["Predicted No", "Predicted Yes", "Total"]
  cm = cm.set_index([["Actual No", "Actual Yes", "Total" ]])
  print("Model Confusion Matrix")
  print(cm)

  print("Model Classification Report")
  print(classification_report(y_test, predictions))

  elapsed_time = (time.time() - start_time)/60
  pct_comp = (count/len(nodes_list2))*100
  print(str(round(pct_comp, 2)) + "% complete" + " (Elapsed time = " + str(round(elapsed_time, 2)) + " min")




model 1/1


84533/84533 [==============================] - 163s 2ms/step - loss: 0.2360 - binary_accuracy: 0.9244 - val_loss: 0.2448 - val_binary_accuracy: 0.9214
Model Confusion Matrix
            Predicted No  Predicted Yes    Total
Actual No        2808574         166738  2975312
Actual Yes        109984         521427   631411
Total            2918558         688165  3606723
Model Classification Report
              precision    recall  f1-score   support

           0       0.96      0.94      0.95   2975312
           1       0.76      0.83      0.79    631411

    accuracy                           0.92   3606723
   macro avg       0.86      0.88      0.87   3606723
weighted avg       0.93      0.92      0.92   3606723

100.0% complete (Elapsed time = 6.42 min


In [ ]:
#lf = MLPClassifier(solver='lbfgs', alpha=1e-5,
#                    hidden_layer_sizes=(5, 2), random_state=1)

#clf.fit(X, y)

In [ ]:
#plot network accuracy vs epoch

In [ ]:
# ignore this
#create list of nodes that should be tried to find best neural network
nodes_list = [10, 20, 30, 40, 50]
roc_outcomes = []
pr_outcomes = []
loss_hx_list = []
model_number = np.array(np.arange(1, len(nodes_list)+1)).tolist()
count = 0
columns_list = ["Epochs"]

#Run neural network for all nodes 
import time 
start_time = time.time()

for nodes in nodes_list:
  count += 1

  try:
    columns_list.append("model " + str(count))

    print("\n")
    print("model " + str(count) + "/" + str(len(nodes_list)))
    print("\n")

    model = Sequential()
    model.add(Dense(units = 23,input_dim=23,activation='relu'))
    model.add(Dense(units = nodes, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    model.fit(X_train, y_train, validation_split = 0.25, epochs=5, batch_size = 32, callbacks = [EarlyStopping(monitor="val_loss", patience=3)])

    #pred_prob = model.predict_proba(X_test)
    #predictions = model.predict_classes(X_test)
    pred_prob=model.predict(X_test)
    predictions=np.argmax(predict_prob,axis=1)

    logit_roc_out = roc_auc_score(y_test, pred_prob)
    average_precision = average_precision_score(y_test, predictions)
    roc_outcomes.append(logit_roc_out)
    pr_outcomes.append(average_precision)

    cm = pd.DataFrame(confusion_matrix(y_test, predictions))
    cm["Total"] = np.sum(cm, axis =1)
    cm = cm.append(np.sum(cm, axis =0), ignore_index =True)
    cm.columns = ["Predicted No", "Predicted Yes", "Total"]
    cm = cm.set_index([["Actual No", "Actual Yes", "Total" ]])
    print("Model Confusion Matrix")
    print(cm)

    print("Model Classification Report")
    print(classification_report(y_test, predictions))

    elapsed_time = (time.time() - start_time)/60
    pct_comp = (count/len(nodes_list))*100
    print(str(round(pct_comp, 2)) + "% complete" + " (Elapsed time = " + str(round(elapsed_time, 2)) + " min")
  except Exception:
    print("Warning")
    pass



model 1/5


Epoch 1/5


KeyboardInterrupt: ignored